In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,23500
2,Huelva,Confirmados PDIA 14 días,3625
3,Huelva,Tasa PDIA 14 días,"706,3935927665297"
4,Huelva,Confirmados PDIA 7 días,1217
5,Huelva,Total Confirmados,23731
6,Huelva,Curados,12411
7,Huelva,Fallecidos,233


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  23500.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6220.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 514 personas en los últimos 7 días 

Un positivo PCR cada 167 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,23500.0,1217.0,3625.0,513170.0,237.153380,706.393593,170.0
Huelva-Costa,14273.0,754.0,2176.0,289548.0,260.405874,751.516156,104.0
Condado-Campiña,6509.0,345.0,1094.0,156231.0,220.826853,700.245150,51.0
Huelva (capital),6220.0,280.0,863.0,143837.0,194.664794,599.984705,44.0
Almonte,760.0,51.0,139.0,24507.0,208.103807,567.184886,17.0
Punta Umbría,626.0,40.0,119.0,15355.0,260.501465,774.991859,15.0
Lepe,1892.0,116.0,309.0,27880.0,416.068867,1108.321377,13.0
Ayamonte,1148.0,90.0,247.0,21104.0,426.459439,1170.394238,12.0
Moguer,767.0,42.0,115.0,21867.0,192.070243,525.906617,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,158.0,32.0,61.0,2885.0,1109.185442,2114.384749,3.0
Berrocal,19.0,3.0,5.0,303.0,990.099010,1650.165017,0.0
Paymogo,71.0,3.0,19.0,1162.0,258.175559,1635.111876,0.0
Villalba del Alcor,436.0,18.0,54.0,3366.0,534.759358,1604.278075,0.0
Isla Cristina,1923.0,97.0,263.0,21393.0,453.419343,1229.374094,4.0
San Bartolomé de la Torre,187.0,4.0,46.0,3761.0,106.354693,1223.078968,0.0
Alájar,15.0,6.0,9.0,759.0,790.513834,1185.770751,0.0
Ayamonte,1148.0,90.0,247.0,21104.0,426.459439,1170.394238,12.0
Palma del Condado (La),956.0,39.0,126.0,10801.0,361.077678,1166.558652,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cerro de Andévalo (El),131.0,1.0,14.0,2327.0,42.973786,601.633004,0.0,0.071429
San Bartolomé de la Torre,187.0,4.0,46.0,3761.0,106.354693,1223.078968,0.0,0.086957
Villarrasa,127.0,1.0,9.0,2211.0,45.228403,407.055631,0.0,0.111111
Jabugo,64.0,1.0,8.0,2260.0,44.247788,353.982301,0.0,0.125000
Cabezas Rubias,28.0,1.0,8.0,706.0,141.643059,1133.144476,0.0,0.125000
Paymogo,71.0,3.0,19.0,1162.0,258.175559,1635.111876,0.0,0.157895
Zalamea la Real,84.0,3.0,17.0,3054.0,98.231827,556.647020,2.0,0.176471
Escacena del Campo,127.0,2.0,10.0,2287.0,87.450809,437.254045,0.0,0.200000
Paterna del Campo,110.0,5.0,25.0,3457.0,144.634076,723.170379,0.0,0.200000
